A ternary search tree has nodes with the following attributes:
* a character, can be `None`;
* a Boolean flag that indicates whether the character represented
  by this node has been the last in a string that was inserted in the
  tree;
* the "less-than" child;
* the "equals" child and
* the "larger-than" child.

The data structure should support the following operations:
* string insert
* string search
* prefix string search
* return the number of strings stored in the data structure
* return all strings stored in the data structure

Also ensure that an instance of the data structure can be visualy represented, e.g., in aSCII format.

In [2]:
import time
import random
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
from typing import List, Tuple

Ternary Search Tree Implementation: \
    *Represents a node in a Ternary Search Tree (TST).\
    *Each node stores a character, a flag indicating if it's the end of a word,\
    *and pointers to left, middle, and right child nodes.

In [3]:
class TSTNode:
    def __init__(self, char):
        self.char = char
        self.is_end = False
        self.left = None
        self.middle = None
        self.right = None

In [4]:

class TernarySearchTree:
    def __init__(self):
        self.root = None

    def insert(self, word):
        if not word:
            return
        self.root = self._insert(self.root, word, 0)

    def _insert(self, node, word, index):
        char = word[index]

        if node is None:
            node = TSTNode(char)

        if char < node.char:
            node.left = self._insert(node.left, word, index)
        elif char > node.char:
            node.right = self._insert(node.right, word, index)
        else:
            if index + 1 < len(word):
                node.middle = self._insert(node.middle, word, index + 1)
            else:
                node.is_end = True
        return node